# Импорты библиотек

In [1]:
import sympy
import pandas
import plotly.express as px
import numpy

# Входные данные

In [2]:
x = sympy.symbols('x')

n = 10
f = x ** 2 - sympy.sin(x)
a = 0.5
b = 1.0

# Определение $x_i$ и $f_i$

In [3]:
h = (b - a) / n
x_points = []
y_points = []
for i in range(n + 1):
    x_points.append(a + i * h)
    y_points.append(float(f.subs(x, x_points[i])))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)
figure.show()

# Расчёт таблицы конечных разностей

Конечная разность n-го порядка рассчитывается по формуле:

$\Delta^n f_i = \Delta^{n - 1} f_{i + 1} - \Delta^{n - 1} f_{i}$

Причём конечная разность 1-го порядка:

$\Delta f_i = f_{i + 1} - f_{i}$

In [4]:
fd_table = [
    x_points,
    y_points
]

for i in range(n):
    fd_table.append([])
    for j in range(n - i):
        fd_table[i + 2].append(fd_table[i + 1][j + 1] - fd_table[i + 1][j])
    for j in range(n - i, n + 1):
        fd_table[i + 2].append(None)

In [5]:
finite_differences_table = pandas.DataFrame({
    "x": x_points,
})

for i in range(n + 1):
    finite_differences_table[f"{i}-й порядок"] = fd_table[i + 1]

finite_differences_table

,x,0-й порядок,1-й порядок,2-й порядок,3-й порядок,4-й порядок,5-й порядок,6-й порядок,7-й порядок,8-й порядок,9-й порядок,10-й порядок
0,0.50,-0.229426,0.009238,0.006306,0.000105,-0.000004,-2.532940e-07,9.450127e-09,6.094865e-10,-2.514966e-11,-1.453004e-12,5.961898e-14
1,0.55,-0.220187,0.015545,0.006411,0.000101,-0.000004,-2.438439e-07,1.005961e-08,5.843369e-10,-2.660266e-11,-1.393385e-12,NaN
2,0.60,-0.204642,0.021956,0.006513,0.000098,-0.000004,-2.337843e-07,1.064395e-08,5.577342e-10,-2.799605e-11,NaN,NaN
3,0.65,-0.182686,0.028469,0.006610,0.000094,-0.000004,-2.231403e-07,1.120168e-08,5.297381e-10,NaN,NaN,NaN
4,0.70,-0.154218,0.035079,0.006704,0.000089,-0.000004,-2.119386e-07,1.173142e-08,NaN,NaN,NaN,NaN
5,0.75,-0.119139,0.041783,0.006793,0.000085,-0.000005,-2.002072e-07,NaN,NaN,NaN,NaN,NaN
6,0.80,-0.077356,0.048576,0.006878,0.000080,-0.000005,NaN,NaN,NaN,NaN,NaN,NaN
7,0.85,-0.028780,0.055453,0.006958,0.000075,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.90,0.026673,0.062411,0.007033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.95,0.089084,0.069445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Интерполяционные формулы для узлов с постоянным шагом

## I формула Ньютона

Рассмотрим $x* \in [a,b]$, который располагается ближе к началу отрезка. Тогда в качестве точки $x_s$ рассмотрим точку, которая находится перед $x*$, остальные точки будут располагаться справа от $x*$ и поочерёдно нумероваться $x_{s + 1}, x_{s + 2}, ..., x_{s + n}$, где $x_{s + n} = b$. Теперь переобозначим точку $x* = x_s + th$, где $0 < t < 1$ и $t = \frac{x* - x_s}{h}$

Тогда получим многочлен:

$\displaystyle L_n(x_s + th) = f_s + t \Delta^1 f_s + \frac{t (t - 1)}{2} \Delta^2 f_s + ... + \frac{t (t - 1) ... (t - (n - 1))}{n!} \Delta^n f_s = f_s + \sum_{i = 1}^{n} \frac{\displaystyle \prod_{j = 0}^{i - 1} (t - j)}{i!} \Delta^i f_s$

In [6]:
def newton_polynomial_1(x_star, start):
    result = 0
    t = (x_star - x_points[start]) / h
    factor = 1

    for i in range(n + 1 - start):
        if i > 0:
            factor *= (t - i + 1) / i

        result += fd_table[i + 1][start] * factor

    return result

## II формула Ньютона

Рассмотрим $x* \in [a,b]$, который располагается ближе к концу отрезка. Тогда в качестве точки $x_s$ рассмотрим точку, которая находится после $x*$, остальные точки будут располагаться слева от $x*$ и поочерёдно нумероваться $x_{s - 1}, x_{s - 2}, ..., x_0$, где $x_0 = a$. Теперь переобозначим точку $x* = x_{s} + th$, где $-1 < t < 0$ и $t = \frac{x* - x_{s}}{h}$

Тогда получим многочлен:

$\displaystyle L_n(x_s + th) = f_s + t \Delta^1 f_{s - 1} + \frac{t (t + 1)}{2} \Delta^2 f_{s - 2} + ... + \frac{t (t + 1) ... (t + (n - 1))}{n!} \Delta^n f_{s - n} = f_s + \sum_{i = 1}^{n} \frac{\displaystyle \prod_{j = 0}^{i - 1} (t + j)}{i!} \Delta^i f_{s - i}$

In [7]:
def newton_polynomial_2(x_star, start):
    result = 0
    t = (x_star - x_points[start]) / h
    factor = 1

    for i in range(start, -1, -1):
        if i < start:
            factor *= (t + start - i - 1) / (start - i)

        result += fd_table[start - i + 1][i] * factor

    return result

## I формула Гаусса

Рассмотрим $x* \in [a,b]$, который располагается ближе к середине, причём ближайшая к нему точка находится слева. Тогда в качестве точки $x_s$ рассмотрим точку, которая находится перед $x*$, остальные точки будут располагаться с чередованием сначала справа от $x*$, потом $-$ слева и поочерёдно нумероваться $x_{s + 1}, x_{s - 1}, x_{s + 2}, x_{s - 2} ..., x_{s + n/2}, x_{s - n/2}$, где $x_{s + n/2} = b$ и/или $x_{s - n/2} = a$. Теперь переобозначим точку $x* = x_{s} + th$, где $0 < t \leq 0.5$ и $t = \frac{x* - x_{s}}{h}$

Тогда получим многочлен:

$\displaystyle L_n(x_s + th) = f_s + t \Delta^1 f_{s} + \frac{t (t - 1)}{2} \Delta^2 f_{s - 1} + ... + \frac{t (t^2 - 1) ... (t^2 - (n/2 - 2)^2) (t - (n/2 - 1))}{n!} \Delta^n f_{s - n/2 + 1} = f_s + \sum_{i = 1}^{n/2} \frac{\displaystyle t (t - i) \prod_{j = 1}^{i - 1} (t^2 - j^2)}{(2i)!} \Delta^{2i} f_{s - i} + \sum_{i = 1}^{n/2 + 1} \frac{\displaystyle t \prod_{j = 1}^{i - 1} (t^2 - j^2)}{(2i - 1)!} \Delta^{2i - 1} f_{s - i - 1}$

In [8]:
def gauss_polynomial_1(x_star, start):
    result = 0
    t = (x_star - x_points[start]) / h
    
    factor = 1
    index = start

    term_count = start * 2 + 1
    if term_count > n + 1:
        term_count = (n - start + 1) * 2 - 1

    for i in range(term_count):
        indent = i // 2 + 1 if i % 2 == 1 else -i // 2

        if i > 0:
            index += 0 if i % 2 == 1 else -1

            term = indent - 1 if i % 2 == 1 else indent
            factor *= (t + term) / i

        result += fd_table[i + 1][index] * factor

    return result

## II формула Гаусса

Рассмотрим $x* \in [a,b]$, который располагается ближе к середине, причём ближайшая к нему точка находится справа. Тогда в качестве точки $x_s$ рассмотрим точку, которая находится после $x*$, остальные точки будут располагаться с чередованием сначала слева от $x*$, потом $-$ справа и поочерёдно нумероваться $x_{s - 1}, x_{s + 1}, x_{s - 2}, x_{s + 2} ..., x_{s - n/2}, x_{s + n/2}$, где $x_{s - n/2} = b$ и/или $x_{s + n/2} = a$. Теперь переобозначим точку $x* = x_{s} + th$, где $-0.5 \leq t < 0$ и $t = \frac{x* - x_{s}}{h}$

Тогда получим многочлен:

$\displaystyle L_n(x_s + th) = f_s + t \Delta^1 f_{s} + \frac{t (t + 1)}{2} \Delta^2 f_{s - 1} + ... + \frac{t (t^2 - 1) ... (t^2 - (n/2 - 2)^2) (t + (n/2 - 1))}{n!} \Delta^n f_{s - n/2 + 1} = f_s + \sum_{i = 1}^{n/2} \frac{\displaystyle t (t + i) \prod_{j = 1}^{i - 1} (t^2 - j^2)}{(2i)!} \Delta^{2i} f_{s - i} + \sum_{i = 1}^{n/2 + 1} \frac{\displaystyle t \prod_{j = 1}^{i - 1} (t^2 - j^2)}{(2i - 1)!} \Delta^{2i - 1} f_{s - i - 1}$

In [9]:
def gauss_polynomial_2(x_star, start):
    result = 0
    t = (x_star - x_points[start]) / h
    
    factor = 1
    index = start

    term_count = start * 2 + 1
    if term_count > n + 1:
        term_count = (n - start + 1) * 2 - 1

    for i in range(term_count):
        indent = i // 2 if i % 2 == 0 else -i // 2

        if i > 0:
            index += -1 if i % 2 == 1 else 0

            term = indent + 1 if i % 2 == 1 else indent
            factor *= (t + term) / i

        result += fd_table[i + 1][index] * factor

    return result

## Формула Стирлинга

Рассмотрим $x* \in [a,b]$, который располагается ближе к середине, причём ближайшая к нему точка находится слева или справа. Тогда в качестве точки $x_s$ рассмотрим точку, которая ближе к $x*$, остальные точки будут располагаться с чередованием в зависимости от того, с какой стороны находится $x_s$, и поочерёдно нумероваться $x_{s - 1}, x_{s + 1}, x_{s - 2}, x_{s + 2} ..., x_{s - n/2}, x_{s + n/2}$, где $x_{s - n/2} = b$ и/или $x_{s + n/2} = a$ или $x_{s + 1}, x_{s - 1}, x_{s + 2}, x_{s - 2} ..., x_{s + n/2}, x_{s - n/2}$, где $x_{s + n/2} = b$ и/или $x_{s - n/2} = a$. Теперь переобозначим точку $x* = x_{s} + th$, где $-0.5 \leq t < 0$ или $0 < t \leq 0.5$ и $t = \frac{x* - x_{s}}{h}$

Тогда получим многочлен:

$\displaystyle L_n(x_s + th) = f_s + t \frac{\Delta^{1} f_{s - 1} + \Delta^{1} f_{s}}{2} + \frac{t^2}{2} \Delta^2 f_{s - 1} + ... + \frac{t (t^2 - 1) ... (t^2 - (n/2 - 2)^2)}{(n - 1)!} \frac{\Delta^{n - 2} f_{s - (n/2 - 2)} + \Delta^{n - 2} f_{s - (n2 - 2) + 1}}{2} + \frac{t^2 (t^2 - 1) ... (t^2 - (n/2 - 2)^2)}{(n)!} \Delta^{n} f_{s - (n/2 - 1)} = f_s + t^2 \sum_{i = 1}^{n/2} \frac{\displaystyle \prod_{j = 1}^{i - 1} (t^2 - j^2)}{(2i)!} \Delta^{2i} f_{s - i} + t \sum_{i = 1}^{n/2 + 1} \frac{\displaystyle \prod_{j = 1}^{i - 2} (t^2 - j^2)}{(2i - 1)!} \frac{\Delta^{2i - 1} f_{s - i} + \Delta^{2i - 1} f_{s - i - 1}}{2}$

In [10]:
def stirling_polynomial(x_star, start, term_count):
    result = fd_table[1][start]
    t = (x_star - x_points[start]) / h
    if abs(t) > 0.5:
        t = float(0.5 * numpy.sign(t))

    odd_factor = t
    even_factor = t ** 2 * 0.5

    for i in range(1, term_count // 2 + 1):
        result += (fd_table[2 * (i - 1) + 2][start - i + 1] + fd_table[2 * (i - 1) + 2][start - i]) * odd_factor * 0.5
        odd_factor *= (t ** 2 - i ** 2) / ((2 * i) * (2 * i + 1))

        result += fd_table[2 * i + 1][start - i] * even_factor
        even_factor *= (t ** 2 - i ** 2) / ((2 * i + 1) * (2 * i + 2))

    return result

## Формула Бесселя

Рассмотрим $x* \in [a,b]$, который располагается ближе к середине, причём ближайшая к нему точка находится слева или справа. Тогда в качестве точки $x_s$ рассмотрим точку слева, остальные точки будут располагаться с чередованием сначала справа от $x*$, потом $-$ слева и поочерёдно нумероваться $x_{s + 1}, x_{s - 1}, x_{s + 2}, x_{s - 2} ..., x_{s + n/2}, x_{s - n/2}$, где $x_{s + n/2} = b$ и/или $x_{s - n/2} = a$. Теперь переобозначим точку $x* = x_{s} + th$, где $0 < t \leq 0.5$ и $t = \frac{x* - x_{s}}{h}$, или $x* = x_{s + 1} + th$, где $-0.5 \leq t < 0$ и $t = \frac{x* - x_{s + 1}}{h}$, в зависимости от того, какая из них ближе

Тогда получим многочлен:

$\displaystyle L_{n + 2}(x_{s / s + 1} + th) = \frac{f_s + f_{s + 1}}{2} + (t - 1/2) \Delta^{1} f_{s} + \frac{t (t - 1)}{2} \frac{\Delta^2 f_{s - 1} + \Delta^2 f_{s}}{2} + ... + \frac{t (t^2 - 1) ... (t^2 - (n/2 - 1)^2) (t - n/2)}{n!} \frac{\Delta^{n} f_{s - n/2} + \Delta^{n} f_{s - n/2 + 1}}{2} + \frac{t (t^2 - 1) ... (t^2 - (n/2 - 1)^2) (t - n/2) (t - 1/2)}{(n + 1)!} \Delta^{n + 1} f_{s - n/2 + 1} = f_s + (t - 1/2) \sum_{i = 1}^{n/2 + 1} \frac{\displaystyle \prod_{j = 0}^{i - 1} (t^2 - j^2)}{t (t + i - 1) (2i - 1)!} \Delta^{2i - 1} f_{s - i + 1} + \sum_{i = 1}^{n/2 + 1} \frac{\displaystyle \prod_{j = 1}^{i} (t^2 - j^2)}{t (t + i) (2i)!} \frac{\Delta^{2i} f_{s - i} + \Delta^{2i} f_{s - i + 1}}{2}$

In [11]:
def bessel_polynomial(x_star, start, term_count):
    result = (fd_table[1][start] + fd_table[1][start + 1]) * 0.5
    t = (x_star - x_points[start]) / h
    if abs(t) > 0.5:
        t = float(0.5 * numpy.sign(t))

    even_factor = t * (t - 1) * 0.5
    odd_factor = t - 0.5

    for i in range(1, term_count // 2):
        result += (fd_table[2 * i + 1][start - i] + fd_table[2 * i + 1][start - i + 1]) * 0.5 * even_factor
        even_factor *= (t ** 2 - i ** 2) / ((2 * i + 1) * (2 * i + 2))

        result += fd_table[2 * i][start - i] * odd_factor
        odd_factor *= (t ** 2 - i ** 2) / ((2 * i) * (2 * i + 1))

    return result

## Выбор формулы

Для выбора формулы изначальный отрезок $[a; b]$ на 3 равные секции. Формулы будут выбираться в зависимости от того, в какой секции расположена $x*$:

**1-я секция:</br>**
В случае, если $x*$ находится в 1-й секции, то используется I формула Ньютона и в качестве точки $x_s$ берётся точка слева от $x*$

**2-я секция:</br>**
В случае, если $0.49 \leq |t| \leq 0.5$, то происходит выбор между формулами Стирлинга и Бесселя. Для Стирлинга в качестве $x_s$ берётся точка, ближайшая к $x*$, для Бесселя $-$ точка слева от $x*$. Затем рассматривается, какое количество узлов может охватить каждая формула, и выбирается та, которая охватывает большее число узлов.</br>
В случае, если $0 < |t| < 0.49$, то в зависимости от знака $t$ выбирается формула Гаусса: если $t$ положительное, то используется I формула Гаусса, а в качестве $x_s$ берётся точка слева от $x*$, в ином случае используется II формула Гаусса, а в качестве $x_s$ берётся точка справа от $x*$

**3-я секция:</br>**
В случае, если $x*$ находится в 3-й секции, то используется II формула Ньютона и в качестве точки $x_s$ берётся точка справа от $x*$

In [12]:
def compute_polynomial(x_star):
    def define_start_point(x_star):
        for i in range(n):
            if x_points[i] <= x_star <= x_points[i + 1]:
                return i

        raise ValueError("x* вне отрезка [a, b]")
    
    def define_section(x_star):
        full_section = h * n
        x_0 = x_points[0]

        if x_star <= full_section / 3 + x_0:
            return 1
        elif full_section / 3 + x_0 <= x_star <=  2 * full_section / 3 + x_0:
            return 2
        elif 2 * full_section / 3 + x_0 <= x_star <= full_section + x_0:
            return 3
        else:
            raise ValueError("x* вне отрезка [a, b]")
        
    def choose_bessel_or_stirling(x_star, start, start2):
        stirling_point_count = start * 2 + 1
        if stirling_point_count > n + 1:
            stirling_point_count = (n - start + 1) * 2 - 1

        left_for_bessel = min(start, start2) + 1
        right_for_bessel = n - max(start, start2) + 1
        bessel_point_count = min(left_for_bessel, right_for_bessel) * 2

        if bessel_point_count >= stirling_point_count:
            return {
                "L_n(x*)": bessel_polynomial(x_star, min(start, start2), bessel_point_count),
                "start": min(start, start2),
                "mode": "B"
            } 
        else:
            return {
                "L_n(x*)": stirling_polynomial(x_star, start, stirling_point_count),
                "start": start,
                "mode": "S"
            }  


    start = define_start_point(x_star)
    section = define_section(x_star)

    if section == 1:
        return {
            "L_n(x*)": newton_polynomial_1(x_star, start),
            "start": start,
            "mode": "N1"
        }
    elif section == 2:
        distance = (x_star - x_points[start]) / h
        
        if 0.49 <= distance <= 0.5:
            return choose_bessel_or_stirling(x_star, start, start + 1)
        elif 0 < distance < 0.49:
            return {
                "L_n(x*)": gauss_polynomial_1(x_star, start),
                "start": start,
                "mode": "G1"
            }
        elif distance > 0.5:
            distance = (x_points[start + 1] - x_star) / h
            if distance > 0.5:
                distance = 0.5

            if 0.49 <= distance <= 0.5:
                return choose_bessel_or_stirling(x_star, start + 1, start)
            elif 0 < distance < 0.49:
                return {
                    "L_n(x*)": gauss_polynomial_2(x_star, start + 1),
                    "start": start + 1,
                    "mode": "G2"
                }
    elif section == 3:
        return {
            "L_n(x*)": newton_polynomial_2(x_star, start + 1),
            "start": start + 1,
            "mode": "N2"
        }
    else:
        raise ValueError("x* вне отрезка [a, b]")

# Вычисление $L_n(x*)$ и $R_n(x*)$

## Инициализация точек $x**, x***, x****$

In [13]:
x_stars = [
    0.52,
    0.97,
    0.73
]

figure = px.line(x=x_points, y=y_points, markers=True, color_discrete_sequence=["rgb(120, 120, 120)"])
figure.update_traces(name="f(x)", showlegend=True)

for i, x_star in enumerate(x_stars):
    figure.add_scatter(
        x=[x_star], 
        y=[float(f.subs(x, x_star))],
        name=f"x{(i + 1) * "*"} = {x_star}"
    )

figure.show()

## Вычисление $(n + 1)$-й производной от $f(x)$ и определение минимума и максимума на отрезке

In [14]:
def compute_f_n_plus_1(degree):
    f_n_plus_1 = sympy.diff(f, x, degree)
    f_n_plus_2 = sympy.diff(f_n_plus_1, x)
    extremes_f_n_plus_1 = sympy.solve(sympy.Eq(f_n_plus_2, 0), x)

    x_f_plus_1 = [x_points[0], x_points[-1]] + [c for c in extremes_f_n_plus_1 if c.is_real and x_points[0] <= c <= x_points[-1]]
    y_f_plus_1 = [float(f_n_plus_1.subs(x, c)) for c in x_f_plus_1]
    f_n_plus_1_min = min(y_f_plus_1)
    f_n_plus_1_max = max(y_f_plus_1)

    print("минимум f^(n + 1) =", f_n_plus_1_min)
    print("максимум f^(n + 1) =", f_n_plus_1_max)
    print("оценка =", f_n_plus_1_max - f_n_plus_1_min)
    print(10 * '-')

    return {
        "max": f_n_plus_1_max,
        "min": f_n_plus_1_min,
    }

## Вычисление $R_n(x)$ и определение его минимума и максимума

In [15]:
def compute_R_n(x_star, polynomial):
    mode = polynomial["mode"]
    start = polynomial["start"]
    omega_n_plus_1 = 1
    f_n_plus_1 = {}

    if mode == "N1":
        f_n_plus_1 = compute_f_n_plus_1(n + 1 - start)

        for i in range(n + 1 - start):
            omega_n_plus_1 *= (x_star - x_points[start + i]) / (i + 1)
    elif mode == "N2":
        f_n_plus_1 = compute_f_n_plus_1(start + 1)

        for i in range(start, -1, -1):
            omega_n_plus_1 *= (x_star - x_points[i]) / (start - i + 1)
    elif mode == "G1":
        term_count = start * 2 + 1
        if term_count > n + 1:
            term_count = (n - start + 1) * 2 - 1

        f_n_plus_1 = compute_f_n_plus_1(term_count)

        for i in range(term_count):
            indent = i // 2 + 1 if i % 2 == 1 else -i // 2
            omega_n_plus_1 *= (x_star - x_points[start + indent]) / (i + 1)
    elif mode == "G2":
        term_count = start * 2 + 1
        if term_count > n + 1:
            term_count = (n - start + 1) * 2 - 1

        f_n_plus_1 = compute_f_n_plus_1(term_count)

        for i in range(term_count):
            indent = i // 2 if i % 2 == 0 else -i // 2
            omega_n_plus_1 *= (x_star - x_points[start + indent]) / (i + 1)
    elif mode == "S":
        term_count = start * 2 + 1
        if term_count > n + 1:
            term_count = (n - start + 1) * 2 - 1

        f_n_plus_1 = compute_f_n_plus_1(term_count)

        for i, element in enumerate(range(start - term_count // 2, start + term_count // 2 + 1)):
            omega_n_plus_1 *= (x_star - x_points[element]) / (i + 1)
    elif mode == "B":
        left_terms = start + 1
        right_terms = n - start
        term_count = min(left_terms, right_terms) * 2

        f_n_plus_1 = compute_f_n_plus_1(term_count)

        for i, element in enumerate(range(start - term_count // 2 + 1, start + term_count // 2 + 1)):
            omega_n_plus_1 *= (x_star - x_points[element]) / (i + 1)
    else:
        print(mode)
        raise(ValueError("Нет такого модификатора"))
    
    R_n = [
        f_n_plus_1["max"] * omega_n_plus_1,
        f_n_plus_1["min"] * omega_n_plus_1
    ]

    print("минимум R_n =", min(R_n))
    print("максимум R_n =", max(R_n))
    print("оценка =", abs(R_n[0] - R_n[1]))

    return R_n

# Заключение

In [16]:
for i, x_star in enumerate(x_stars):
    polynomial = compute_polynomial(x_star)
    L_n = polynomial["L_n(x*)"]

    figure = px.scatter(
        x=[x_star], 
        y=[float(f.subs(x, x_star))], 
        title=f"Оценка качества L_n(x{"*" * (i + 2)})")
    figure.update_traces(name=f"f(x{"*" * (i + 2)})", showlegend=True)
    figure.add_scatter(x=[x_star], y=[L_n], name=f"L_n(x{"*" * (i + 2)})")
    figure.show()

    R_n_list = compute_R_n(x_star, polynomial)
    R_n_min = min(R_n_list)
    R_n_max = max(R_n_list)

    R_n = L_n - float(f.subs(x, x_star))

    print(f"Условие: {R_n_min} < {R_n} < {R_n_max}")
    print(R_n_min < R_n < R_n_max)

минимум f^(n + 1) = 0.5403023058681398
максимум f^(n + 1) = 0.8775825618903728
оценка = 0.337280256022233
----------
минимум R_n = 2.5341440291402287e-17
максимум R_n = 4.11606721855231e-17
оценка = 1.581923189412081e-17
Условие: 2.5341440291402287e-17 < 3.885780586188048e-16 < 4.11606721855231e-17
False


минимум f^(n + 1) = 0.5403023058681398
максимум f^(n + 1) = 0.8775825618903728
оценка = 0.337280256022233
----------
минимум R_n = -2.61515250633824e-17
максимум R_n = -1.610074072492685e-17
оценка = 1.0050784338455552e-17
Условие: -2.61515250633824e-17 < -9.71445146547012e-17 < -1.610074072492685e-17
False


минимум f^(n + 1) = 0.5403023058681398
максимум f^(n + 1) = 0.8775825618903728
оценка = 0.337280256022233
----------
минимум R_n = 2.966065150385984e-19
максимум R_n = 4.817612335055881e-19
оценка = 1.851547184669897e-19
Условие: 2.966065150385984e-19 < 5.551115123125783e-17 < 4.817612335055881e-19
False
